## **Install Packages**

In [24]:
pip install langgraph langchain-core langchain-community langchain huggingface_hub langchain-google-genai gradio sentence-transformers faiss-cpu git+https://github.com/openai/whisper.git ffmpeg-python gTTS

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ied2fq62
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-ied2fq62
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# **LITERATURE**

The following mental health workflow model has the following characteristics:

**FEATURES:**
- model = gemini-1.5-flash free (It was chosen because it is compatible with Langchain)
- embedding = all-MiniLM-L6-v2
- vectore_store = FAISS
- average processing time = 25 seconds

**ADDITIONAL:**
2. Prompt Engineer = added a more detailed description of the topic to generate warm responses without losing professionalism
3. Graph = implemented a workflow graph in HTML to view the outcome

**Gemini Free LIMITATION:**
- RATE LIMITS: 15 RPM
- 1,000,000 TPM
- 1,500 RPD
- PRICE (INPUT): Free of charge
- PRICE (OUTPUT): Free of charge
- GROUNDING WITH GOOGLE SEARCH: Not available

## **Load Packages and Libraries**

In [25]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated, Optional
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import AIMessage
from dotenv import load_dotenv
from google import genai
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import drive, userdata
import logging
import pandas as pd
import kagglehub
import gradio as gr
import os

In [26]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


## **Mount Google Drive**

In [27]:
# Mount Google Drive
try:
    drive.mount('/content/drive')
    logger.info("Google Drive mounted successfully")
except Exception as e:
    logger.error(f"Failed to mount Google Drive: {str(e)}")
    raise


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **LOAD MODELS**

In [28]:
# Load environment variables
route = "/content/drive/MyDrive/OMDENA/MENTAL/gemini/.env"
if os.path.exists(route):
    load_dotenv(dotenv_path=route)
    logger.info(f".env file loaded from {route}")
else:
    logger.warning(f".env file not found at {route}. Falling back to Colab secrets.")

# Get API key (try .env first, then Colab secrets)
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    try:
        api_key = userdata.get('GOOGLE_API_KEY')
        logger.info("API key loaded from Colab secrets")
    except:
        logger.error("GOOGLE_API_KEY not found in .env or Colab secrets")
        raise ValueError(
            "GOOGLE_API_KEY is required. Please set it in /content/drive/MyDrive/OMDENA/MENTAL/gemini/.env "
            "or add it to Colab secrets under the name 'GOOGLE_API_KEY'. "
            "Get an API key from https://console.cloud.google.com/apis/credentials"
        )

# Initialize LLM
try:
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0, google_api_key=api_key)
    logger.info("ChatGoogleGenerativeAI initialized successfully")
except Exception as e:
    logger.error(f"Failed to initialize ChatGoogleGenerativeAI: {str(e)}")
    raise


## **LOAD DATA**

In [29]:
# Load existing datasets
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGbYKgw5x2xteCHSjLhkY5FHTPjtVnvBkN_5m2p6clfdUJK77CLDRSdq5RbPygNygaTFsK3xghrfi4/pub?output=csv'
try:
    df1 = pd.read_csv(url)
    logger.info("Google Sheets dataset loaded")
except Exception as e:
    logger.error(f"Failed to load Google Sheets dataset: {str(e)}")
    raise

path = kagglehub.dataset_download("emirhanai/social-media-usage-and-emotional-well-being")
path_ = "/kaggle/input/social-media-usage-and-emotional-well-being"
try:
    df2 = pd.read_csv(f"{path_}/train.csv")
    logger.info("Kaggle dataset loaded")
except Exception as e:
    logger.error(f"Failed to load Kaggle dataset: {str(e)}")
    raise

# Load new dataset (mental_health_cases.csv)
# new_dataset_path = "/content/sample_data/mental_health_cases.csv"
# try:
#     df3 = pd.read_csv(new_dataset_path)
#     logger.info("Mental health cases dataset loaded")
# except Exception as e:
#     logger.error(f"Failed to load mental_health_cases.csv: {str(e)}")
#     raise

# Convert rows to documents
def create_documents(df, content_column=None):
    documents = []
    for index, row in df.iterrows():
        if content_column:
            content = str(row[content_column])
        else:
            content = "".join(str(value) for value in row)
        documents.append(Document(page_content=content))
    return documents

# Create documents from all datasets
docs1 = create_documents(df1, content_column="Comments")
docs2 = create_documents(df2)
all_docs = docs1 + docs2
logger.info(f"Created {len(all_docs)} documents from datasets")


## **CODE**

In [31]:
# Define graph state
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    age: int
    gender: str
    language: Optional[str]
    emotion: str
    query: str
    docs: list
    next: str

# Initialize FAISS vector store
def create_faiss_db(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
    split_docs = splitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_documents(split_docs, embeddings)
    vector_store.save_local("/content/drive/MyDrive/OMDENA/MENTAL/faiss_index")
    logger.info("FAISS vector store created and saved")
    return vector_store

def get_vector_store(documents):
    faiss_index_path = "/content/drive/MyDrive/OMDENA/MENTAL/faiss_index"
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    if os.path.exists(faiss_index_path):
        vector_store = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
        logger.info("Loaded existing FAISS vector store")
    else:
        vector_store = create_faiss_db(documents)
    return vector_store

# Node: InputCollector
def input_collector(state: GraphState) -> GraphState:
    return state

# Node: ValidateInput
def validate_input(state: GraphState) -> dict:
    if state['age'] > 0 and state['gender'] in ['male', 'female'] and state['emotion']:
        return {"next": "RetrieveDocs"}
    return {"next": END}

# Node: RetrieveDocs
def retrieve_docs(state: GraphState) -> GraphState:
    vector_store = get_vector_store(all_docs)
    retriever = vector_store.as_retriever()
    docs = retriever.get_relevant_documents(state["query"])
    state["docs"] = docs
    logger.info(f"Retrieved {len(docs)} documents for query: {state['query']}")
    return state

# Node: SystemReplyGenerator
def generate_reply(state: GraphState) -> GraphState:
    base_context = """
You are a virtual emotional wellness agent designed to help people understand and improve their mental health by providing empathetic, warm, and professional responses to various emotions. Respond with accessible and understandable strategies and support tailored to the emotions indicated by the user.

# Overview
For each emotion mentioned by users, respond with empathy and provide practical strategies that can help them manage that emotion in a healthy way. Maintain a warm but professional tone, using emoticons sparingly to convey humanity without losing professionalism.

# Specific Details
1. **Tone**: Empathetic, friendly, and encouraging, but avoid being too casual.
2. **Helpful Content**: Combination of emotional support (encouraging words) and practical strategies (techniques, activities, small actions).
3. **Emoticons**: Use them sparingly (1-2 maximum per response) according to the supportive tone.
4. **Personalized Strategies**: Make sure you tailor your recommendations to the specific emotion indicated by the user.
5. **Accessible Language**: Make sure your explanations and advice are clear, simple, and free of complicated technical terms.

# Possible Emotions to Manage
- Stress
- Anxiety
- Depression
- Sadness
- Overwhelm
- Exhaustion
- Anger
- Feeling Lonely
- Confusion
- Neutral/Unsure

# Output Format
Write a response for each emotion with a warm tone, structured in the following parts:
1. **Emotional Acknowledgment**: Acknowledge how the user feels, validating their emotions.
2. **Emotional Support**: Show empathy and provide words of encouragement.
3. **Practical Actions**: Suggest concrete strategies (at least 2-3) they can put into practice, incorporating relevant suggestions from retrieved documents where applicable.
4. **Positive Closing**: End with an encouraging statement.

# Notes
- Incorporate specific suggestions from retrieved documents (e.g., comments from mental_health_cases.csv) when they align with the user's emotion and query.
- Ensure responses are varied and not repetitive, using retrieved documents as additional context.
- Maintain a genuine tone and avoid clinical diagnoses.
    """
    prompt = (
        f"User (Age: {state['age']}, Gender: {state['gender']}, Emotion: {state['emotion']}): {state['query']}\n"
        f"Generate a response based on all the instructions in the base context with the information in {state['docs']}. "
        f"The response must incorporate relevant suggestions from the retrieved documents, especially from the mental health dataset, when applicable."
    )
    language = state.get("language", "English")
    if language.lower() != "english":
        prompt += f" Translate the final answer to {language}."
    try:
        response = llm.invoke(prompt)
        reply = response.content if hasattr(response, "content") else response
        state["messages"].append(AIMessage(content=reply))
        logger.info("Generated response successfully")
    except Exception as e:
        logger.error(f"Failed to generate response: {str(e)}")
        reply = "I'm sorry, I couldn't generate a response. Please try again."
        state["messages"].append(AIMessage(content=reply))
    if any(term in reply.lower() for term in ["stress", "focus", "overwhelmed"]):
        state["next"] = "relevant"
    else:
        state["next"] = "irrelevant"
    return state

# Node: RewriteReply
def rewrite_reply(state: GraphState) -> GraphState:
    previous_reply = state['messages'][-1].content if state['messages'] else ""
    prompt = f"Rewrite this reply to be more aligned with the user's emotional state ({state['emotion']}): {previous_reply}"
    try:
        response = llm.invoke(prompt)
        reply = response.content if hasattr(response, "content") else response
        state["messages"].append(AIMessage(content=reply))
        logger.info("Rewrote response successfully")
    except Exception as e:
        logger.error(f"Failed to rewrite response: {str(e)}")
        reply = "I'm sorry, I couldn't rewrite the response. Please try again."
        state["messages"].append(AIMessage(content=reply))
    state["next"] = "relevant"
    return state

# Node: ConversationHandler
def conversation_handler(state: GraphState) -> dict:
    latest = state['messages'][-1].content if state['messages'] else ""
    if any(x in latest.lower() for x in ["exit", "quit", "bye"]):
        return {"next": END}
    return {"next": "FollowUpAnalyzer"}

# Node: FollowUpAnalyzer
def follow_up_analyzer(state: GraphState) -> GraphState:
    return state

# Build the LangGraph
workflow = StateGraph(GraphState)
workflow.add_node("InputCollector", RunnableLambda(input_collector))
workflow.add_node("ValidateInput", RunnableLambda(validate_input))
workflow.add_node("RetrieveDocs", RunnableLambda(retrieve_docs))
workflow.add_node("SystemRepliesGenerator", RunnableLambda(generate_reply))
workflow.add_node("RewriteReply", RunnableLambda(rewrite_reply))
workflow.add_node("ConversationHandler", RunnableLambda(conversation_handler))
workflow.add_node("FollowUpAnalyzer", RunnableLambda(follow_up_analyzer))

# Edges
workflow.set_entry_point("InputCollector")
workflow.add_edge("InputCollector", "ValidateInput")
workflow.add_conditional_edges("ValidateInput", lambda x: x["next"])
workflow.add_edge("RetrieveDocs", "SystemRepliesGenerator")
workflow.add_conditional_edges("SystemRepliesGenerator", lambda x: x["next"])
workflow.add_edge("RewriteReply", "ConversationHandler")
workflow.add_conditional_edges("ConversationHandler", lambda x: x["next"])
workflow.add_edge("FollowUpAnalyzer", "RetrieveDocs")

# Compile
app = workflow.compile()

# Gradio Interface
def run_chatbot(age, gender, emotion, query):
    try:
        input_state = {
            "age": int(age),
            "gender": gender,
            "emotion": emotion,
            "query": query,
            "messages": [],
            "docs": []
        }
        result = app.invoke(input_state)
        reply = result["messages"][-1].content if result["messages"] else "No response generated."
        return reply
    except Exception as e:
        logger.error(f"Chatbot error: {str(e)}")
        return f"⚠️ Error occurred: {str(e)}"

gr.Interface(
    fn=run_chatbot,
    inputs=[
        gr.Number(label="Age"),
        gr.Radio(choices=["male", "female"], label="Gender"),
        gr.Dropdown(
            choices=[
                "stress", "anxiety", "depression", "sadness", "overwhelmed",
                "burnout", "anger", "lonely", "confused", "neutral / unsure"
            ],
            label="Emotion"
        ),
        gr.Textbox(lines=3, label="What’s on your mind?")
    ],
    outputs="text",
    title="🧠 GreenLife Health Chatbot 🤖",
    description="Fill in your details and ask what's bothering you. The chatbot will offer empathetic support and suggestions."
).launch()

# Example usage
if __name__ == "__main__":
    initial_state = {
        "age": 50,
        "gender": "female",
        "emotion": "stress",
        "query": "I'm feeling very overwhelmed with work and can't focus.",
        "messages": [],
        "docs": []
    }
    final_result = app.invoke(initial_state)
    print(final_result["messages"][-1].content)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bb19d2727adb65d8ce.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


I understand you're feeling overwhelmed and unable to focus at work.  That's incredibly stressful, especially at age 50.  While the provided documents discuss internet addiction and obsessive-compulsive disorder,  your description doesn't directly suggest either of those conditions.  However, feeling overwhelmed and unable to focus can be a symptom of various things, including stress itself, and possibly even burnout.

It's important to address this.  Here are a few suggestions:

* **Prioritize tasks:**  Try breaking down your workload into smaller, more manageable tasks.  Focus on completing one at a time, rather than feeling overwhelmed by the whole picture.  This can help improve focus.

* **Time management techniques:** Explore techniques like the Pomodoro Technique (working in focused bursts with short breaks) to improve concentration and prevent burnout.

* **Stress reduction techniques:**  Incorporate stress-reducing activities into your daily routine. This could include exercis

**SPANISH**


In [32]:
from langchain_core.messages import HumanMessage
initial_state = {
    "age": 28,
    "gender": "hombre",
    "emotion": "ansioso",
    "query": "Me siento triste, acabo de romper con mi novia",
    "messages": [],
    "docs": [],
    "language":"spanish"
}

final_result = app.invoke(initial_state)
print("Final Response:", final_result)
#print(final_result["messages"][-1].content)

Final Response: {'messages': [], 'age': 28, 'gender': 'hombre', 'language': 'spanish', 'emotion': 'ansioso', 'query': 'Me siento triste, acabo de romper con mi novia', 'docs': [], 'next': '__end__'}


## **GRAPH VISUALIZATION**

In [ ]:
# !apt-get install -y graphviz libgraphviz-dev
# !pip install pygraphviz langgraph[visualizations]

In [33]:
from langchain_core.runnables.graph_mermaid import MermaidDrawMethod
# Extrae el código Mermaid desde tu objeto
mermaid_code = app.get_graph().draw_mermaid()

# Muestra el código Mermaid en la consola
print(mermaid_code)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__(<p>__start__</p>)
	InputCollector(InputCollector)
	ValidateInput(ValidateInput)
	RetrieveDocs(RetrieveDocs)
	SystemRepliesGenerator(SystemRepliesGenerator)
	RewriteReply(RewriteReply)
	ConversationHandler(ConversationHandler)
	FollowUpAnalyzer(FollowUpAnalyzer)
	__end__(<p>__end__</p>)
	InputCollector --> ValidateInput;
	__start__ --> InputCollector;
	ValidateInput --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



## **Speech to Text**

In [34]:
import tempfile
import os
import whisper
whisper_model = whisper.load_model("base")  # you can also try "small" or "medium"

## **Text to Speech**

In [35]:
from gtts import gTTS
import tempfile
import gradio as gr

def run_chatbot_with_audio(age, gender, emotion, query_text, audio_path):
    try:
        # Transcribe if audio is present
        if audio_path:
            transcription = whisper_model.transcribe(audio_path)
            query_text = transcription["text"].strip()

        if not query_text:
            return "⚠️ Please speak or type your message.", None

        # Prepare chatbot input
        input_state = {
            "age": int(age),
            "gender": gender,
            "emotion": emotion,
            "query": query_text,
            "messages": [],
            "docs": []
        }

        # Generate response
        result = app.invoke(input_state)
        reply = result["messages"][-1].content if result["messages"] else "No response generated."

        # Convert reply to speech
        tts = gTTS(reply)
        tts_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
        tts.save(tts_file.name)

        return f"🗣️ You said: \"{query_text}\"\n🤖 Chatbot: {reply}", tts_file.name

    except Exception as e:
        return f"⚠️ Error occurred: {str(e)}", None


In [36]:
gr.Interface(
    fn=run_chatbot_with_audio,
    inputs=[
        gr.Number(label="Age"),
        gr.Radio(choices=["male", "female"], label="Gender"),
        gr.Dropdown(
            choices=[
                "stress", "anxiety", "depression", "sadness", "overwhelmed",
                "burnout", "anger", "lonely", "confused", "neutral / unsure"
            ],
            label="Emotion"
        ),
        gr.Textbox(lines=3, label="📝 Or type your message"),
        gr.Audio(label="🎤 Or speak (mic/upload)", type="filepath", format="wav")
    ],
    outputs=[
        gr.Textbox(label="Chatbot Reply"),
        gr.Audio(label="🔊 Chatbot Speaks")
    ],
    title="🧠 GreenLife Health Chatbot 🎙️",
    description="Type or speak your thoughts. The chatbot will respond with empathy and voice."
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e726489517f4ed9315.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
